In [1]:
import numpy as np
import yaml
from astropy.table import Table
from astropy.io import fits
_bitdefs = yaml.load("""
#- target pixel mask
targetmask:
    - [QSOLYA,       0, "Lyman alpha QSO"]
    - [QSO,       1, "Tracer QSO"]
    - [LRG,      2, "Luminous Red Galaxy"]
    - [ELG, 3, "Emission Line Galaxy"]
    - [STAR,    4, "Standard Star"]
    - [SKY, 5, "Sky"] 
    - [BGS, 6, "Bright Galaxy"]
""")

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
# reusing https://github.com/desihub/desispec/blob/master/py/desispec/maskbits.py
#- Class to provide mask bit utility functions
class BitMask(object):
    """BitMask object.
    """
    def __init__(self, name, bitdefs):
        """
        Args:
            name : name of this mask, must be key in bitdefs
            bitdefs : dictionary of different mask bit definitions each value is a list of [bitname, bitnum, comment]

        Users are not expected to create BitMask objects directly.

        See maskbits.ccdmask, maskbits.specmask, maskbits.fibermask, ...
        """
        self._name = name
        self._bitname = dict()  #- key num -> value name
        self._bitnum = dict()   #- key name -> value num
        self._comment = dict()  #- key name or num -> comment
        for bitname, bitnum, comment in bitdefs[name]:
            assert bitname not in self._bitnum
            assert bitnum not in self._bitname
            self._bitnum[bitname] = bitnum
            self._bitname[bitnum] = bitname
            self._comment[bitname] = comment
            self._comment[bitnum] = comment

    def bitnum(self, bitname):
        """Return bit number (int) for bitname (string)"""
        return self._bitnum[bitname]

    def bitname(self, bitnum):
        """Return bit name (string) for this bitnum (integer)"""
        return self._bitname[bitnum]

    def comment(self, bitname_or_num):
        """Return comment for this bit name or bit number"""
        return self._comment[bitname_or_num]

    def mask(self, name_or_num):
        """Return mask value, i.e. 2**bitnum for this name or number"""
        if isinstance(name_or_num, int):
            return 2**name_or_num
        else:
            return 2**self._bitnum[name_or_num]

    def names(self, mask=None):
        """Return list of names of masked bits.
        If mask=None, return names of all known bits.
        """
        names = list()
        if mask is None:
            for bitnum in sorted(self._bitname.keys()):
                names.append(self._bitname[bitnum])
        else:
            bitnum = 0
            while bitnum < mask:
                if (2**bitnum & mask):
                    if bitnum in self._bitname.keys():
                        names.append(self._bitname[bitnum])
                    else:
                        names.append('UNKNOWN'+str(bitnum))
                bitnum += 1

        return names

    #- Allow access via mask.BITNAME
    def __getattr__(self, name):
        if name in self._bitnum:
            return 2**self._bitnum[name]
        else:
            raise AttributeError('Unknown mask bit name '+name)

    #- What to print
    def __repr__(self):
        result = list()
        result.append( self._name+':' )
        for i in sorted(self._bitname.keys()):
            result.append('    - [{:16s} {:2d}, "{}"]'.format(self._bitname[i]+',', i, self._comment[i]))

        return "\n".join(result)

In [3]:
targetmask = BitMask("targetmask", _bitdefs)
n_tiles = 10
target_per_tile = 25000
total_n_targets = target_per_tile * n_tiles / 2
random_ID_list =  np.arange(total_n_targets, dtype='int64')
random_ra_list = np.random.random(total_n_targets)
random_dec_list = np.random.random(total_n_targets)
random_type_A_list = np.int_(np.random.random(total_n_targets)*7)
random_type_B_list = np.int_(np.random.random(total_n_targets)*7)
print np.size(np.where(random_type_A_list == random_type_B_list))

17724


# TARGET FORMAT

In [4]:
def write_target_fits(id_list, ra_list, dec_list, type_list, filename):
    t = Table([id_list, ra_list, dec_list, type_list], names=('ID', 'RA', 'DEC', 'TYPE'), meta={'name': 'target table'})
    t.write(filename, format='fits', overwrite=True)

def read_target_fits(filename):
    t = Table.read(filename, format='fits')
    return t
    
for i_tile in range(n_tiles):
    filename = 'tmp/targets_%06d.fits'%(i_tile)
    tile_id = np.random.choice(random_ID_list, size=target_per_tile)
    tile_ra = random_ra_list[tile_id]
    tile_dec = random_dec_list[tile_id]
    tile_type_A = random_type_A_list[tile_id]
    tile_type_B = random_type_B_list[tile_id] 
    tile_type = np.ones(target_per_tile, dtype='int64')
    
    for i in range(target_per_tile):
        tile_type[i]  = targetmask.mask(random_type_A_list[i]) | targetmask.mask(random_type_B_list[i])
    write_target_fits(tile_id, tile_ra, tile_dec, tile_type, filename)
    t = read_target_fits(filename)